In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import napari
from scipy.sparse import issparse
from scipy.stats import ranksums, false_discovery_control
from tqdm import tqdm
import pickle
import pandas as pd
from anndata import AnnData, read_h5ad
import scanpy as sc

In [2]:
%matplotlib qt

In [3]:
def compute_pearson_residuals(X, theta=100.0, clip=None, 
                              copy=False, return_params=False):
    """from dynamo-release"""

    """Compute Pearson residuals from count data.

    Pearson residuals are a measure of the deviation of observed counts from expected counts under a Poisson or negative
    binomial model.

    Args:
        X: array_like count matrix, shape (n_cells, n_genes).
        theta: the dispersion parameter for the negative binomial model. Must be positive.
        clip: The maximum absolute value of the residuals. Residuals with absolute value larger than `clip` are clipped
            to `clip`. If `None`,`clip` is set to the square root of the number of cells in `X`.
        check_values: whether to check if `X` contains non-negative integers. If `True` and non-integer values are
            found, a `UserWarning` is issued.
        copy: whether to make a copy of `X`.

    Returns:
        The Pearson residuals.
    """
    X = X.copy() if copy else X

    # check theta
    if theta <= 0:
        # TODO: would "underdispersion" with negative theta make sense?
        # then only theta=0 were undefined..
        raise ValueError("Pearson residuals require theta > 0")
    # prepare clipping
    if clip is None:
        n = X.shape[0]
        clip = np.sqrt(n)
    if clip < 0:
        raise ValueError("Pearson residuals require `clip>=0` or `clip=None`.")

    if issparse(X):
        sums_genes = np.sum(X, axis=0)
        sums_cells = np.sum(X, axis=1)
        sum_total = np.sum(sums_genes).squeeze()
    else:
        sums_genes = np.sum(X, axis=0, keepdims=True)
        sums_cells = np.sum(X, axis=1, keepdims=True)
        sum_total = np.sum(sums_genes)

    mu = np.array(sums_cells @ sums_genes / sum_total)
    diff = np.array(X - mu)
    residuals = diff / np.sqrt(mu + mu**2 / theta)

    # clip
    residuals = np.clip(residuals, a_min=-clip, a_max=clip)
    
    if return_params:
        sigma = np.sqrt(mu + mu**2 / theta)
        return residuals, mu, sigma
    else:
        return residuals


def bin_aps(aps, bins):
    _counts, bins = np.histogram(aps, bins)
    bins = bins[1:]
    binned_aps = np.zeros_like(aps)
    for i in range(len(binned_aps)):
        binned_aps[i] = get_ap_bin(aps[i], bins)
    
    return binned_aps
    

def get_ap_bin(this_ap, bins):
    this_bin = np.where(np.abs(this_ap - bins) == np.nanmin(np.abs(this_ap - bins)))[0][0]

    return this_bin



In [5]:
"""load the anndata file"""
file_path = r'/media/brandon/Data2/Brandon/fly_immune/Flysta3d/L3_b_count_normal_stereoseq.h5ad'
f = h5py.File(file_path, 'r')
f.keys()

<KeysViewHDF5 ['X', 'layers', 'obs', 'obsm', 'uns', 'var']>

In [6]:
"""extract the raw reads and processes"""
raw_reads =  np.array(f['layers'].get('raw_counts'))

# filter reads to 5% detection
detection_percent = np.sum(raw_reads > 0, axis=0) / len(raw_reads)
filter_sel = detection_percent > 0.05
filtered_reads = raw_reads[:, filter_sel]

# convert to pearson residuals
#residuals = compute_pearson_residuals(filtered_reads, theta=10)


In [7]:
"""get selection of just fat body cells"""
a = np.array(f['obs'].get('annotation'))
sel = (a == 4)

In [170]:
adata = read_h5ad(file_path)

In [9]:
fb = adata[sel, filter_sel].copy()

In [10]:
sc.tl.pca(fb)

In [49]:
sc.pl.pca_variance_ratio(fb, n_pcs=50, log=True)

In [11]:
sc.pp.neighbors(fb)

/home/brandon/anaconda3/envs/scanpy/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
sc.tl.umap(fb)

In [13]:
sc.tl.leiden(fb, n_iterations=2)

/tmp/ipykernel_3332730/106880534.py:1: FutureWarning: In the future, the default backend for leiden will be igraph instead of leidenalg.

 To achieve the future defaults please pass: flavor="igraph" and n_iterations=2.  directed must also be False to work with igraph's implementation.
  sc.tl.leiden(fb, n_iterations=2)


In [14]:
sc.pl.umap(fb, color='leiden')

In [67]:
sc.pl.umap(fb, color=["TotA"])

In [21]:
sc.pl.umap(fb, color=["new_y"], cmap='viridis')

In [18]:
# parameters for ap bins
n_y_bins = 10
ys = np.array(fb.obs['new_y'])
y_bins = np.linspace(np.min(ys), np.max(ys), n_y_bins)
binned_ys = bin_aps(ys, y_bins)

In [27]:
y_bins

array([-178.21475  ,  -87.6591625,    2.896425 ,   93.4520125,
        184.0076   ])

In [19]:
fb.obs['binned_y'] = binned_ys

In [176]:
sc.pl.umap(fb, color=["binned_y"], cmap='viridis')

In [51]:
# for g in fb.var.index:
#     print(g)

In [115]:
fb.var

""
geneID
128up
14-3-3epsilon
14-3-3zeta
18SrRNA-Psi:CR41602
26-29-p
...
zetaTry
zf30C
zip


## Identify differentially expressed genes as markers

In [61]:
res = 0.02
sc.tl.leiden(
        fb, key_added=f"leiden_res_{res:4.2f}", resolution=res, flavor="igraph"
    )

In [63]:
sc.pl.umap(fb, color='leiden_res_0.02')

In [64]:
sc.tl.rank_genes_groups(fb, groupby="leiden_res_0.02", method="wilcoxon")

In [65]:
sc.pl.rank_genes_groups_dotplot(
    fb, groupby="leiden_res_0.02", standard_scale="var", n_genes=5
)

## Let's try to compute pearson residuals by hand

In [126]:
filtered_data = adata[:, filter_sel].copy()

In [127]:
filtered_data.X = compute_pearson_residuals(filtered_data.layers['raw_counts'], theta=50)

/tmp/ipykernel_3557772/4140725622.py:47: RuntimeWarning: invalid value encountered in divide
  residuals = diff / np.sqrt(mu + mu**2 / theta)


In [217]:
plt.figure()
plt.plot(np.nanmean(filtered_data.layers['raw_counts'], axis=0), np.nanvar(filtered_data.X, axis=0), 'ko', alpha=0.2)
plt.xscale('log')
plt.yscale('linear')
plt.xlabel('mean count')
plt.ylabel('var pearson residual')


Text(0, 0.5, 'var pearson residual')

In [148]:
res_fb = filtered_data[sel].copy()

In [149]:
sc.tl.pca(res_fb)
sc.pp.neighbors(res_fb)
sc.tl.umap(res_fb)
sc.tl.leiden(res_fb, n_iterations=2, resolution=0.03, flavor='igraph')

In [133]:
sc.pl.umap(res_fb, color='leiden')

In [150]:
sc.tl.rank_genes_groups(res_fb, groupby='leiden', method="wilcoxon", corr_method='bonferroni')
#sc.tl.filter_rank_genes_groups(res_fb, groupby='leiden', min_fold_change=1, compare_abs=True, min_in_group_fraction=0.25)

/home/brandon/anaconda3/envs/scanpy/lib/python3.12/site-packages/scanpy/tools/_rank_genes_groups.py:455: RuntimeWarning: invalid value encountered in log2
  self.stats[group_name, "logfoldchanges"] = np.log2(
/home/brandon/anaconda3/envs/scanpy/lib/python3.12/site-packages/scanpy/tools/_rank_genes_groups.py:455: RuntimeWarning: invalid value encountered in log2
  self.stats[group_name, "logfoldchanges"] = np.log2(
/home/brandon/anaconda3/envs/scanpy/lib/python3.12/site-packages/scanpy/tools/_rank_genes_groups.py:455: RuntimeWarning: invalid value encountered in log2
  self.stats[group_name, "logfoldchanges"] = np.log2(
/home/brandon/anaconda3/envs/scanpy/lib/python3.12/site-packages/scanpy/tools/_rank_genes_groups.py:455: RuntimeWarning: invalid value encountered in log2
  self.stats[group_name, "logfoldchanges"] = np.log2(
/home/brandon/anaconda3/envs/scanpy/lib/python3.12/site-packages/scanpy/tools/_rank_genes_groups.py:455: RuntimeWarning: invalid value encountered in log2
  self.st

In [151]:
sc.tl.filter_rank_genes_groups(res_fb, groupby='leiden', min_fold_change=0, compare_abs=True, min_in_group_fraction=0.25)

TypeError: 'NoneType' object is not subscriptable

In [155]:
sc.pl.rank_genes_groups_heatmap(
    res_fb, groupby='leiden', n_genes=10, key='rank_genes_groups', vmin=0, vmax=30, show_gene_labels=True)

In [106]:
np.min(res_fb.X)

-6.922497

In [124]:
# parameters for ap bins
n_y_bins = 5
ys = np.array(fb.obs['new_y'])
y_bins = np.linspace(np.min(ys), np.max(ys), n_y_bins)
binned_ys = bin_aps(ys, y_bins)

res_fb.obs['binned_y'] = binned_ys

In [125]:
sc.pl.umap(res_fb, color='binned_y')

In [55]:
sc.pl.umap(res_fb, color=["DptA"], vmin=0, vmax=10)

## Differentially expressed genes as markers, from pearson residuals

In [78]:
res = 0.05
sc.tl.leiden(
        res_fb, key_added=f"leiden_res_{res:4.2f}", resolution=res, flavor="igraph"
    )

In [79]:
sc.pl.umap(res_fb, color=f"leiden_res_{res:4.2f}")

In [82]:
sc.tl.rank_genes_groups(res_fb, groupby="leiden_res_0.02", method="wilcoxon")

/home/brandon/anaconda3/envs/scanpy/lib/python3.12/site-packages/scanpy/tools/_rank_genes_groups.py:455: RuntimeWarning: invalid value encountered in log2
  self.stats[group_name, "logfoldchanges"] = np.log2(
/home/brandon/anaconda3/envs/scanpy/lib/python3.12/site-packages/scanpy/tools/_rank_genes_groups.py:455: RuntimeWarning: invalid value encountered in log2
  self.stats[group_name, "logfoldchanges"] = np.log2(
/home/brandon/anaconda3/envs/scanpy/lib/python3.12/site-packages/scanpy/tools/_rank_genes_groups.py:455: RuntimeWarning: invalid value encountered in log2
  self.stats[group_name, "logfoldchanges"] = np.log2(


In [83]:
sc.pl.rank_genes_groups_dotplot(
    res_fb, groupby="leiden_res_0.05", standard_scale="var", n_genes=5
)

categories: 0, 1, 2, etc.
var_group_labels: 0, 1, 2


In [85]:
res_fb

AnnData object with n_obs × n_vars = 6628 × 6505
    obs: 'slice_ID', 'raw_x', 'raw_y', 'new_x', 'new_y', 'new_z', 'annotation', 'leiden', 'binned_y', 'leiden_res_0.02', 'leiden_res_0.50', 'leiden_res_2.00', 'leiden_res_0.05'
    uns: 'annotation_colors', 'pca', 'neighbors', 'umap', 'leiden', 'leiden_colors', 'leiden_res_0.02', 'leiden_res_0.02_colors', 'leiden_res_0.50', 'leiden_res_2.00', 'leiden_res_2.00_colors', 'leiden_res_0.50_colors', 'leiden_res_0.05', 'leiden_res_0.05_colors', 'rank_genes_groups', 'dendrogram_leiden_res_0.05'
    obsm: 'X_umap', 'spatial', 'X_pca'
    varm: 'PCs'
    layers: 'raw_counts'
    obsp: 'distances', 'connectivities'

## Scanpy's pearson residuals

In [230]:
# overwrite the layer you give it
adata2 = adata[sel, filter_sel].copy()
adata2.layers['log1p'] = adata2.X
adata2.X = sc.experimental.pp.normalize_pearson_residuals(adata2, layer='raw_counts', theta=50, inplace=False)['X']

In [231]:
adata2

AnnData object with n_obs × n_vars = 6628 × 6505
    obs: 'slice_ID', 'raw_x', 'raw_y', 'new_x', 'new_y', 'new_z', 'annotation'
    uns: 'annotation_colors'
    obsm: 'X_umap', 'spatial'
    layers: 'raw_counts', 'log1p'

In [221]:
plt.figure()
plt.plot(np.nanmean(adata2.layers['raw_counts'], axis=0), np.nanvar(adata2.X, axis=0), 'ko', alpha=0.2)
plt.xscale('log')
plt.yscale('linear')
plt.xlabel('mean count')
plt.ylabel('var pearson residual')


Text(0, 0.5, 'var pearson residual')

In [232]:
sc.tl.pca(adata2)
sc.pp.neighbors(adata2)
sc.tl.umap(adata2)
sc.tl.leiden(adata2, n_iterations=2, resolution=0.04, flavor='igraph')

In [239]:
sc.pl.umap(adata2, color='leiden')

In [242]:
# parameters for ap bins
n_y_bins = 5
ys = np.array(adata2.obs['new_y'])
y_bins = np.linspace(np.min(ys), np.max(ys), n_y_bins)
binned_ys = bin_aps(ys, y_bins)

adata2.obs['binned_y'] = [str(int(b)) for b in binned_ys]
adata2.obs['binned_y'] = adata2.obs['binned_y'].astype('category')

In [235]:
sc.pl.umap(adata2, color='binned_y')

In [248]:
sc.pl.umap(adata2, color='new_z')

In [236]:
sc.tl.rank_genes_groups(adata2, groupby='leiden', method="wilcoxon", corr_method='bonferroni', layer='log1p')

In [189]:
sc.tl.filter_rank_genes_groups(adata2, groupby='leiden', min_fold_change=1, min_in_group_fraction=0, max_out_group_fraction=1)

In [238]:
sc.pl.rank_genes_groups_heatmap(
    adata2, groupby='leiden', key='rank_genes_groups', vmin=0, vmax=30, show_gene_labels=True, min_logfoldchange=1)

In [244]:
sc.tl.rank_genes_groups(adata2, groupby='binned_y', key_added='rank_genes_groups_y', method="wilcoxon", corr_method='bonferroni', layer='log1p')

In [245]:
sc.pl.rank_genes_groups_heatmap(
    adata2, groupby='binned_y', key='rank_genes_groups_y', vmin=0, vmax=30, show_gene_labels=True, min_logfoldchange=1)

## Revisiting DE analysis along AP using scanpy workflow

In [80]:
fb = adata[sel, filter_sel].copy()

In [69]:
ys = np.array(fb.obs['new_y'])
y_bins = np.linspace(np.min(ys), np.max(ys), 5)
# combine bin centers 2 and 3
#y_bins = np.delete(y_bins, 3)
binned_ys = bin_aps(ys, y_bins)
fb.obs['binned_y'] = [str(int(b)) for b in binned_ys]
fb.obs['binned_y'] = fb.obs['binned_y'].astype('category')
y_bins = y_bins[1:]

In [70]:
y_bins

array([-87.6591625,   2.896425 ,  93.4520125, 184.0076   ])

In [81]:
# create the neighbor matrix and umap
sc.tl.pca(fb)
sc.pp.neighbors(fb)
sc.tl.umap(fb)

In [72]:
sc.tl.rank_genes_groups(fb, groupby="binned_y", method="wilcoxon", corr_method='bonferroni')

In [73]:
sc.tl.filter_rank_genes_groups(fb, groupby='binned_y', min_fold_change=1, compare_abs=True, min_in_group_fraction=0.25)

In [51]:
sc.pl.umap(fb, color="binned_y")

In [74]:
sc.pl.rank_genes_groups_heatmap(
    fb, groupby="binned_y", n_genes=10, key='rank_genes_groups_filtered')

In [82]:
res = 0.02
sc.tl.leiden(
        fb, key_added=f"leiden_res_{res:4.2f}", resolution=res, flavor="igraph"
    )

In [86]:
fb

AnnData object with n_obs × n_vars = 6628 × 6505
    obs: 'slice_ID', 'raw_x', 'raw_y', 'new_x', 'new_y', 'new_z', 'annotation', 'leiden_res_0.02'
    uns: 'annotation_colors', 'pca', 'neighbors', 'umap', 'leiden_res_0.02'
    obsm: 'X_umap', 'spatial', 'X_pca'
    varm: 'PCs'
    layers: 'raw_counts'
    obsp: 'distances', 'connectivities'

In [87]:
sc.tl.rank_genes_groups(fb, groupby=f"leiden_res_{res:4.2f}", method="wilcoxon", corr_method='bonferroni')

In [88]:
sc.tl.filter_rank_genes_groups(fb, groupby=f"leiden_res_{res:4.2f}", min_fold_change=1, compare_abs=True, min_in_group_fraction=0.25)

In [89]:
sc.pl.rank_genes_groups_heatmap(
    fb, groupby=f"leiden_res_{res:4.2f}", n_genes=10, key='rank_genes_groups_filtered')

In [90]:
sc.pl.umap(fb, color=f"leiden_res_{res:4.2f}")

In [249]:
import napari

In [253]:
points = adata2.obsm['spatial']
points[:, [0,2]] = points[:, [2,0]]

In [257]:
viewer = napari.Viewer()
viewer.add_points(points, size=2)
viewer.layers[0].scale = (2.5, 1, 1)

In [269]:
viewer.layers[0].scale = (2.5, 1, 1)

In [263]:
adata2.obs['slice_ID']

L3_b_S01_5700x86600      L3_b_S01
L3_b_S01_5850x86600      L3_b_S01
L3_b_S01_5900x86600      L3_b_S01
L3_b_S01_5900x86700      L3_b_S01
L3_b_S01_5950x86450      L3_b_S01
                           ...   
L3_b_S16_17800x115800    L3_b_S16
L3_b_S16_17800x115850    L3_b_S16
L3_b_S16_17850x115700    L3_b_S16
L3_b_S16_17850x115750    L3_b_S16
L3_b_S16_17850x115850    L3_b_S16
Name: slice_ID, Length: 6628, dtype: category
Categories (16, object): ['L3_b_S01', 'L3_b_S02', 'L3_b_S03', 'L3_b_S04', ..., 'L3_b_S13', 'L3_b_S14', 'L3_b_S15', 'L3_b_S16']

In [267]:
np.min(points[:, 0])

0.7

In [273]:
import pandas as pd

In [276]:
points_df = pd.DataFrame({'z': points[:,0], 'y': points[:,1], 'x':points[:,2], 'cluster':adata2.obs['leiden'].values})

In [281]:
for p in points_df.cluster.unique():
    these_points = points_df[points_df.cluster == p].get(['z', 'y', 'x']).values
    viewer.add_points(these_points, size=2)

In [283]:
for layer in viewer.layers:
   layer.scale = (5, 1, 1)